### Some Note

`./BreastCancer/Data_RNASeq2.mat` is the information datset as `mat` format

- `geneRNASeq2RawMatrix0/1` is for activity level info

- `gNamePureListRNASeq2` show the Genomic name

In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from utils.data_fetch import data_fetch

In [2]:
raw_data = sio.loadmat('./BreastCancer/Data_RNASeq2.mat')
pthway = data_fetch(filepath='./Gene_DATA/sourcePathway.txt')

In [3]:
namelist_free = raw_data['SampleList0']
namelist_cancer = raw_data['SampleList1']
geneNamelist = raw_data['gNamePureListRNASeq2'][0]
activ_free = raw_data['geneRNASeq2RawMatrix0']
activ_cancer = raw_data['geneRNASeq2RawMatrix1']

protein_list = pthway.Namelist

if 'protein' not in pthway.node_className:
    raise NameError('The pathway data does not include the protein info')

# Obtained the data that only inside of protein_list    
protein_list = protein_list[protein_list['GenomeType'] == 'protein']['GenomeName'].values
geneNamelist = [np.array2string(geneNamelist[i]).replace("['",'').replace("']",'') for i in range(geneNamelist.size)]
geneNamelist = np.array(geneNamelist).astype(np.object)

included_gene = np.in1d(geneNamelist, protein_list)

# Get two input data in the end, 
# TODO reshuffle required 
activ_free = activ_free[included_gene]
activ_cancer = activ_cancer[included_gene]

In [4]:
# Here we get the new genenNamelist that only include the contained protein name
geneNamelist = geneNamelist[included_gene]

included_protein = np.in1d(protein_list, geneNamelist)

remained_protein = protein_list[included_protein]
otherpth_list = pthway.Namelist[pthway.Namelist['GenomeType'] != 'protein']['GenomeName'].values
remained_protein_all = np.concatenate([remained_protein, otherpth_list])

In [5]:
# save the temporary edgelist
Edgelist = pthway.Edgelist
Namelist_l = list(remained_protein_all)
Edgelist_l = list(Edgelist.iloc[:,0].values)
Edgelist_ll = list(Edgelist.iloc[:,1].values)
exclude_list = []
for idx, (elem, elem2) in enumerate(zip(Edgelist_l, Edgelist_ll)):
    if ((elem not in Namelist_l) or (elem2 not in Namelist_l)):
        exclude_list.append(idx)

Edgelist = Edgelist.drop(exclude_list).reset_index(drop=True)

In [6]:
# shuffle the data to be the same order of both protein and genome
tracked_index = np.argsort(geneNamelist)
sorted_geneNamelist = geneNamelist[tracked_index]
activ_free = activ_free[tracked_index]
activ_cancer = activ_cancer[tracked_index]

In [7]:
remained_protein = np.sort(remained_protein)

In [34]:
protein_df = pd.DataFrame({'GenomeType': 'protein',
             'GenomeName': remained_protein})
overall_df = pd.concat([protein_df, pthway.Namelist[pthway.Namelist['GenomeType'] != 'protein']],
                      axis=0,ignore_index=True)

In [35]:
# Start to index label
overall_df = overall_df.sort_values(by='GenomeName').reset_index(drop=True)
overall_df = overall_df[~overall_df.duplicated()]
le = LabelEncoder()
le.fit(overall_df['GenomeName'].values)
le.classes_.sort()
edge_index = le.transform(Edgelist.iloc[:,:2].values.reshape(-1)).reshape(-1,2)
all_elem_className = list(le.classes_)


In [13]:
# Label edge_class
le = LabelEncoder()
le.fit(Edgelist['edgeType'])
edge_class = le.transform(Edgelist['edgeType'])
edge_className = list(le.classes_)

# Label node class
le = LabelEncoder()
le.fit(overall_df['GenomeType'])
node_class = le.transform(overall_df['GenomeType'])
node_className = list(le.classes_)        